## Multilayer perceptron, harjoitustyö 2022

Harjoitusdatan lähde: https://www.openml.org/search?type=data&sort=runs&status=active&id=6

P. W. Frey and D. J. Slate. "Letter Recognition Using Holland-style Adaptive Classifiers". Machine Learning 6(2), 1991

Tässä harjoitustyössä pyritään ennustamaan aakkosia niihin liittyvien mustavalkoarvojen mukaan käyttäen neuroverkkomenetelmää. 

In [61]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [43]:
X, y = fetch_openml("letter", version=1, as_frame=True, return_X_y=True)

In [46]:
X

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,2.0,4.0,4.0,3.0,2.0,7.0,8.0,2.0,9.0,11.0,7.0,7.0,1.0,8.0,5.0,6.0
1,4.0,7.0,5.0,5.0,5.0,5.0,9.0,6.0,4.0,8.0,7.0,9.0,2.0,9.0,7.0,10.0
2,7.0,10.0,8.0,7.0,4.0,8.0,8.0,5.0,10.0,11.0,2.0,8.0,2.0,5.0,5.0,10.0
3,4.0,9.0,5.0,7.0,4.0,7.0,7.0,13.0,1.0,7.0,6.0,8.0,3.0,8.0,0.0,8.0
4,6.0,7.0,8.0,5.0,4.0,7.0,6.0,3.0,7.0,10.0,7.0,9.0,3.0,8.0,3.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5.0,10.0,5.0,8.0,3.0,4.0,10.0,7.0,8.0,12.0,10.0,9.0,2.0,9.0,2.0,6.0
19996,4.0,7.0,6.0,5.0,3.0,7.0,8.0,2.0,10.0,12.0,6.0,8.0,1.0,9.0,6.0,8.0
19997,4.0,8.0,4.0,6.0,4.0,7.0,8.0,7.0,4.0,10.0,7.0,6.0,3.0,9.0,3.0,7.0
19998,4.0,11.0,4.0,8.0,3.0,0.0,2.0,4.0,6.0,1.0,0.0,7.0,0.0,8.0,0.0,8.0


Muodostetaan datan käsittelyä varten pipelinet, ja columntransformer, jotka yhdessä muuntavat datan 0 keskiarvoisiksi, keskihajonnalla 1. Pipeline onehotencodaisi kategoriset muuttujat jos niitä olisi, mutta tässä datassa koko data olikin numeerista. Se on osa pipelineä harjoituksen vuoksi.

In [53]:
numerical_transformer = Pipeline(
    steps = [("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps = [("imputer", SimpleImputer(strategy="constant", fill_value = "not_available")),
           ("ohe", OneHotEncoder(handle_unknown = "ignore"))]
)

preprocessor = ColumnTransformer(
    transformers = [("numerical", numerical_transformer, selector(dtype_exclude="category")),
                    ("categorical", categorical_transformer, selector(dtype_include="category"))],
    n_jobs=-1
)

In [82]:
mlp = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", MLPClassifier(max_iter=400, hidden_layer_sizes = [200,200]))]
)

Seuraavaksi fitataan malli, ja ristiinvalidoidaan sen osumatarkkuus.

In [83]:
mlp.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000251CF5C0040>),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='not_available',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000251CF6D3A00>)])),
                ('classifier',
                 MLPClassifier(hidden_layer_sizes=[200, 200], max_iter=400))])

In [84]:
np.mean(cross_val_score(mlp, X,y))

0.9686

Seuraavaksi etsitään ristiinvalidoinnin avulla paras hyperparametrikombinaatio seuraavasta hyperparametriavaruudesta.

In [ ]:
param_grid = {
    "classifier__hidden_layer_sizes": [50,100,200],
    "classifier__activation": ["relu", "logistic", "tanh"],
    "classifier__solver": ["lbfgs", "sgd", "adam"],
    "classifier__learning_rate": ["constant","adaptive"]   
}

grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs =-1)
grid_search.fit(X,y)

In [68]:
grid_search.best_params_

{'classifier__activation': 'relu',
 'classifier__hidden_layer_sizes': 200,
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'adam'}

In [78]:
np.mean(cross_val_score(grid_search, X, y))

C:\Users\Johannes\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Johannes\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Johannes\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Johannes\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and 

0.95845

Päädytään oletusparametreihin.